# 실습 WEEK10: 동적 웹크롤링

:::{admonition} 안내 
:class: info  

- 실습 문제는 사전, 도전, 완전 실습 문제로 구성되어 있습니다.
- 사전 실습 문제는 수업 전에 미리 풀어보시면 도움이 됩니다.
- **<font color='blue'>도전 실습 문제는 etl>우리강좌>코딩수업>에 접속한 후, 해당 주차 문제의 소스코드를 제출하여 테스트를 통과해야 과제 점수를 받을 수 있습니다.</font>**
- **<font color='red'>제출기한은 실습수업시간부터 다음 실습수업 전날 자정까지</font>**입니다.
- 완전 실습 문제는 보너스 실습 문제로 생각하시면 됩니다.  도전 실습 문제를 다 풀어본 학생들이 추가적으로 풀어보시면 도움이 됩니다.
- 사전 실습 문제와 완전 실습 문제는 수업 시간에 별도 풀이를 하지 않습니다.
- 질문은 사전, 도전, 완전 그리고 수업과 관련된 모든 것에  대하여 교수자 or 조교 or 튜터에게 하시면 됩니다.
- **<font color='red'>가독성을 위해 입출력 데이터에서 콜론(:) 다음에는 항상 한칸 공백이 있습니다.</font>**

:::

### 환경설정

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

### 웹브라우저 한글폰트 깨짐 방지
- 코랩에서 웹브라우저를 구동시켜 웹페이지에 접속했을 때 한글 폰트가 깨져서 네모로 나오는 것을 방지한다.

In [ ]:
!apt-get install fonts-unfonts-core 

### 크롬드라이버 설치
- 코랩에서 크롬드라이버(웹드라이버)를 동작시키기 위해 크롬 드라이버를 설치한다.

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

### 셀레니움 설치
- 동적 웹크롤링/스크랩핑을 위한 모듈인 셀레니움을 설치한다.

In [ ]:
!pip install selenium

### 크롬드라이버 구동

- 설치가 성공적으로 완료되었다면 크롬 드라이버의 옵션 몇 가지를 설정해줘야 한다. 
- `options.add_argument("--headless")`는 코랩에서는 윈도우 탭을 열 수 없으므로 윈도우 탭을 생성하지 않겠다는 옵션이다.
- `options.add_argument("--no-sandbox")`는 보안기능을 비활성화하는 옵션이다. 일부 사이트에 접근할 때 웹브라우저에 보안기능이 적용되서 팝업을 차단시킨다거나, 웹사이트 접근을 차단하는 등의 기능이 적용되기도 한다. 이러한 기능이 적용되면 웹페이지에 접속할 수 없으므로 이 기능을 비활성화 시킨다.
- 웹드라이버(크롬브라우저)를 구동시켜서 변수 driver에 저장한다.
  -  `driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)`
- 여기까지 진행하면 실제 브라우저가 구동되는 것이 보이지는 않지만 내부적으로 브라우저가 구동된다.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless") 
options.add_argument("--no-sandbox")

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

## [도전 실습 문제]-함께 풀어보는 문제

### Problem : 서울시 구별 주유소 가격정보 얻기

😄한국석유공사인 오피넷(http://www.opinet.co.kr) 사이트의 싼 주유소 찾기 > 지역별 메뉴에서 서울시 각 구별(강남구~중랑구)까지 모든 주유소의 보통 휘발유 가격정보가 담겨있는 엑셀파일을 모두 다운로드 받으시오.

- 1-1부터 1-6 진행하는 사이에 에러 발생시 driver.close()한 후, 크롬드라이버 구동부터 다시 실행해주세요.

#### 1-1: 😄 사이트 접속하기
오피넷(http://www.opinet.co.kr) 사이트의 싼 주유소 찾기 > 지역별 메뉴에 접속하여 'screenshot_001.png'로 스크린샷을 찍으세요.


#### 1-2: 😄 시/도 입력 받기
시/도를 입력 받아 주유소 > 지역 > 옵션 선택 박스에서 입력 받은 지역이 선택되게 한 후, 'screenshot_002.png'로 스크린샷을 찍으세요.

<div align="center"><img src="https://haesunbyun.github.io/common/images/selenium4.png" style="width:700px;"></div>

Input Sample
```
전남
```

Output Sample
```
시/도: 
True
```

- find_element(), find_elements() 함수의 괄호안에는 추출할 옵션의 종류는 다음과 같다.
- 아래의 옵션을 사용하려면 By 클래스를 먼저 import해야 한다.
  - `from selenium.webdriver.common.by import By`
```
By.ID	태그의 id값으로 추출
By.NAME	태그의 name값으로 추출
By.XPATH	태그의 경로로 추출
By.LINK_TEXT	링크 텍스트값으로 추출
By.PARTIAL_LINK_TEXT	링크 텍스트의 자식 텍스트 값을 추출
By.TAG_NAME	태그명으로 추출
By.CLASS_NAME	태그의 클래스명으로 추출
By.CSS_SELECTOR	css선택자로 추출
```



#### 1-3: 😄 시/군/구 입력받기
시/군/구를 입력 받아 주유소 지역의 옵션 선택 박스에서 입력 받은 지역이 선택되게 한 후, 'screenshot_003.png'로 스크린샷을 찍으세요. 

이때 1-2 문제에서 입력한 값(시/도)에 있는 시/군/구를 입력해야 합니다.

<div align="center"><img src="https://haesunbyun.github.io/common/images/selenium5.png" style="width:700px;"></div>

<font color='blue'>`test case-1`</font> 
---
Input Sample
```
장성군
```

Output Sample
```
시/군/구: 
True
```

#### 1-4: 😄 모든 구 리스트로 저장하기
서울의 모든 구를 변수 sigungu_list에 리스트로 저장하세요.

<font color='blue'>`test case-1`</font> 

Output Sample
```
['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']
```



#### 1-5: 😄 조회 /  엑셀저장 버튼 클릭하기
1-4에서 만든 sigungu_list를 활용하여 옵션 선택 박스에서 '강남구'가 선택되게 한 후, 조회 버튼과 엑셀저장 버튼이 클릭되게 하세요. '강남구'의 인덱스는 0입니다.

<div align="center"><img src="https://haesunbyun.github.io/common/images/selenium6.png" height="500" width="300"style="width:700px;"></div>

#### 1-6: 😄 반복적으로 클릭시키기
반복문을 통해 전체 구 주유소정보를 엑셀파일로 가져오세요.(웹서버의 부하를 주지 않게 하기 위해서 5개의 구만 가져와주세요.)

- 엑셀파일이 다운로드 되어 만들어질 때 까지 기다려야 합니다.
- 엑셀로 다운로드 했을 때 파일이름이 동일하게 저장되어 이전 다운로드 받았던 파일을 덮어씌웁니다. 그래서 파일명을 바꾸어줘야 합니다.
- 코드는 다음과 같습니다.

```python
  while True:
      if '지역_위치별(주유소).xls' in os.listdir(): #os.listdir(): 현재 폴더의 파일명을 리스트로 반환
          break
      else:
          time.sleep(1)
  os.rename('지역_위치별(주유소).xls', f'{gu}.xlsx') 
```


## [도전 실습 문제]-스스로 풀어보는 문제

### Problem: 네이버의 쇼핑에서 검색하여 웹크롤링하기
😄 검색어를 입력받아 네이버의 쇼핑탭(https://shopping.naver.com/home) 에서 검색되게 한 후, 1페이지에 나오는 상품가운데, 최고가와 최저가 금액을 출력하시오.

<font color='blue'>`test case-1`</font> 

Input Sample
```
건조기
```
Output Sample
```
검색상품명: 
48개 검색되었습니다.
최고가: 1656060원
최저가: 29900원
```

#### 1-1: 😄 사이트 접속하기
네이버의 쇼핑탭(https://shopping.naver.com/home) 접속하여 'screenshot_001.png'로 스크린샷을 찍으시오.


True

#### 1-2: 😄 검색어 입력받기
검색상품명을 입력받아 검색어 입력박스에 입력되게 한 후, 검색 클릭버튼까지 클릭한 후 'screenshot_002.png'로 스크린샷을 찍으시오.

<font color='blue'>`test case-1`</font> 

Input Sample
```
건조기
```

Output Sample
```
검색상품명: 
```
<div align="center"><img src="https://haesunbyun.github.io/common/images/selenium7.png" height="250" width="500"style="width:400px;"></div>


#### 1-3: 😄 스크롤링 하기
네이버의 쇼핑페이지는 마우스로 페이지를 스크롤링하거나 스크롤바를 내리면 상품 목록이 나오도록 되어 있습니다. 스크롤링하는 코드는 다음과 같습니다.
아래 코드는 윈도우의 최상단으로부터 3500픽셀까지 내리는 코드입니다. 숫자를 조정하시면 됩니다.

```python
driver.execute_script("window.scrollTo(0, 3500)")
time.sleep(3)
```
스크롤링 된 화면을 'screenshot_003.png'로 스크린샷을 찍으시오.

<font color='blue'>`test case-1`</font> 

Output Sample
```

```
<div align="center"><img src="https://haesunbyun.github.io/common/images/selenium8.png" height="400" width="500" style="width:700px;"></div>


#### 1-4: 😄 상품 가격 찾아서 최고가, 최저가 출력하기

<font color='blue'>`test case-1`</font> 

Output Sample
```
48개 검색되었습니다.
최고가: 1656060원
최저가: 29900원
```
